In [1]:
# from src.datasets import patientDataset, eegDataset
# from src.resnet import ResNet1d
# from src.lstm import Lstm
from tqdm import tqdm
import numpy as np
import mne
from helper_code import *
import matplotlib.pyplot as plt

In [7]:
import numpy as np
from scipy import signal

def psd(eeg_data, fs):
    # Define frequency bands of interest (in Hz)
    freq_bands = {'delta': [0.5, 4],
              'theta': [4, 8],
              'alpha': [8, 12],
              'beta': [12, 30]}

    # Define sampling frequency (in Hz)
    fs = 60

    # Compute PSD and extract power in each frequency band for each epoch
    freqs, psds = signal.welch(eeg_data, fs=fs, nperseg=fs*2)
    band_powers = {}
    for band, freq_range in freq_bands.items():
        freq_mask = (freqs >= freq_range[0]) & (freqs < freq_range[1])
        band_powers[band] = psds[:, freq_mask]
        
    return np.concatenate([band_powers[band] for band in freq_bands])

def bsr(eeg_data, sfreq):
    epoch_duration = 1
    freq_band = [0.5, 25]
    bsr_threshold = 0.5
    nyquist_freq = sfreq / 2
    b, a = signal.butter(4, [freq_band[0] / nyquist_freq, freq_band[1] / nyquist_freq], btype='bandpass')
    filtered_data = signal.filtfilt(b, a, eeg_data)

    epoch_samples = int(epoch_duration * sfreq)
    n_epochs = filtered_data.shape[-1] // epoch_samples
    epochs = filtered_data[:, :n_epochs * epoch_samples].reshape(18, n_epochs, epoch_samples)
    
    rms_amplitude = np.sqrt(np.mean(epochs ** 2, axis=-1))
    median_rms = np.median(rms_amplitude)
    bsr = (rms_amplitude < bsr_threshold * median_rms).mean(axis=-1) * 100
    return bsr


In [11]:
import torch
from torch.utils.data import Dataset
from helper_code import *
import numpy as np, os, sys
import librosa
import mne
import torchaudio.transforms as T

from tqdm import tqdm

n_fft = 2048
hop_length = 512
n_mels = 256
n_mfcc = 256


def featurise_recording(location):
    channels = ['Fp1-F7', 'F7-T3', 'T3-T5', 'T5-O1', 'Fp2-F8', 'F8-T4', 
                'T4-T6', 'T6-O2', 'Fp1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 
                'Fp2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'Fz-Cz', 'Cz-Pz']
    recording_data, sr, cur_channels = load_recording(location)
    signal_data = reorder_recording_channels(recording_data, cur_channels, channels)
    
    resampled_data = librosa.resample(y=signal_data, orig_sr=sr, target_sr=60)
    resampled_data = resampled_data[:, 120*60: 140*60]
#     final2 = psd(resampled_data, 60)
    final1 = bsr(resampled_data, 60)
#     final3 = dwt(resampled_data, 60)
    
    final = np.concatenate((final1, final2))
#     print(final.shape)
    
    return final

def featurise_locs(locs):
    fin = []
    fin.append(featurise_recording(locs[0]))
    fin.append(featurise_recording(locs[len(locs)//2]))
    fin.append(featurise_recording(locs[-1]))
    fin = np.array(fin).flatten()
#     print(fin.shape)
    return fin

def featurise_labels(label):
#     print(label)
    tp = np.array([0,0])
    tp[label] = 1
    return tp

class patientDataset(Dataset):
    def __init__(self, data_folder, segs):
        
        self.no_of_segments = segs
        
        self.data_folder  = data_folder
        self.patient_ids  = find_data_folders(data_folder)
        self.num_patients = len(self.patient_ids)
        
        self.inputs = []
        self.outcomes = []
        
        for i in tqdm(range(self.num_patients)):
            inp, out = self.getitemx(i)
#             if inp.shape[0] == self.no_of_segments:
            self.inputs.append(inp)
            self.outcomes.append(out)
#         self.inputs = np.array(self.inputs)
#         self.outcomes = np.array(self.outcomes)
        
    def getMetadata(self, idx):
        # Load data.
        patient_id = self.patient_ids[idx]
        
        # Define file location.
        patient_metadata_file = os.path.join(self.data_folder, patient_id, patient_id + '.txt')
        recording_metadata_file = os.path.join(self.data_folder, patient_id, patient_id + '.tsv')

        # Load non-recording data.
        patient_metadata = load_text_file(patient_metadata_file)
        recording_metadata = load_text_file(recording_metadata_file)
        
        return patient_metadata, recording_metadata
        
    def getitemx(self, index):
        
        patient_metadata, recording_metadata = self.getMetadata(index)
        
        # Load recordings.
        recording_ids = list(get_recording_ids(recording_metadata))
        
        recording_locations = []
        for recording_id in reversed(recording_ids):
            if recording_id != 'nan':
                recording_location = os.path.join(self.data_folder, self.patient_ids[index], recording_id)
                recording_locations.append(recording_location)
            
            if len(recording_locations) >= self.no_of_segments:
                break
        recording_locations.reverse()
        print(len(recording_locations), get_outcome(patient_metadata))
        return featurise_locs(recording_locations), featurise_labels(get_outcome(patient_metadata))
    
    def __getitem__(self, index):
        
        return self.inputs[index], self.outcomes[index]
        
    def __len__(self):
        return self.num_patients

In [12]:
import pywt
import numpy as np

def dwt(eeg_signal, sfreq):
    eeg_signal = eeg_signal[:, 170*sfreq:180*sfreq]
    # Define the wavelet family and level of decomposition
    wavelet = 'db4'
    level = 5

    # Compute the wavelet coefficients for each channel
    coeffs_list = []
    for ch_signal in eeg_signal:
        coeffs = pywt.wavedec(ch_signal, wavelet, level=level)
        coeffs_list.append(coeffs)

    # Extract the coefficients corresponding to each frequency band for each channel
    delta_coeffs_list = []
    theta_coeffs_list = []
    alpha_coeffs_list = []
    beta_coeffs_list = []
    for coeffs in coeffs_list:
        delta_coeffs = coeffs[level]
        theta_coeffs = coeffs[level-1]
        alpha_coeffs = coeffs[level-2]
        beta_coeffs = coeffs[level-3]
        delta_coeffs_list.append(delta_coeffs)
        theta_coeffs_list.append(theta_coeffs)
        alpha_coeffs_list.append(alpha_coeffs)
        beta_coeffs_list.append(beta_coeffs)

    # Stack the coefficient arrays for each frequency band across channels
    delta_coeffs = np.stack(delta_coeffs_list, axis=0)
    theta_coeffs = np.stack(theta_coeffs_list, axis=0)
    alpha_coeffs = np.stack(alpha_coeffs_list, axis=0)
    beta_coeffs = np.stack(beta_coeffs_list, axis=0)
    
    fin = np.concatenate( (delta_coeffs, theta_coeffs, alpha_coeffs, beta_coeffs), axis=1 ).mean(axis=0)
#     print(fin.shape)
    return fin


In [13]:
from torch.utils.data import DataLoader
import torch

no_of_segs = 82
pattrainset = patientDataset("../train/", no_of_segs)

pattestset = patientDataset("../split_5/", no_of_segs)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

  0%|          | 0/486 [00:00<?, ?it/s]

65 0
(18, 20)


NameError: name 'final2' is not defined

In [ ]:
trainloader = DataLoader(dataset=pattrainset, batch_size=32, shuffle=True, num_workers=20)
valloader = DataLoader(dataset=pattestset, batch_size=32, shuffle=True, num_workers=20)

In [ ]:
X, y = np.array(pattrainset.inputs), np.array(pattrainset.outcomes)
# X = X.reshape((X.shape[0], X.shape[1]*X.shape[2]))
y = y[:,0]
X.shape,y.shape

In [ ]:
import pandas as pd
df = pd.DataFrame(X)
df['y'] = y

In [ ]:
import h2o
from h2o.automl import H2OAutoML
# We will be using default parameter Here with H2O init method
h2o.init()

In [ ]:
# convert pandas DataFrame into H2O Frame
train = h2o.H2OFrame(df)

In [ ]:
X, y = np.array(pattrainset.inputs), np.array(pattrainset.outcomes)
# X = X.reshape((X.shape[0], X.shape[1]*X.shape[2]))
y = y[:,0]
df_val = pd.DataFrame(X)
df_val['y'] = y

test = h2o.H2OFrame(df_val)
x = test.columns
y = 'y'
# remove label classvariable from feature variable
x.remove(y)

In [ ]:
# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

# Run AutoML for 20 base models
aml = H2OAutoML(max_models=20, seed=1)
aml.train(y = y, training_frame = train, leaderboard_frame = test)

In [ ]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

In [ ]:
preds = aml.predict(test)

In [ ]:
preds['p1']

In [ ]:
def validate(model, val_loader, criterion):
    val_loss = []
    val_accs = []
    model.eval()
    for i, (inputs, labels) in enumerate(val_loader):
        inputs = torch.Tensor(inputs)
        inputs = inputs.type(torch.FloatTensor).to(device)
        labels = torch.Tensor(labels)
        labels = labels.type(torch.FloatTensor).to(device)
        
        outputs = model(inputs)
        loss    = criterion(outputs, labels)
        pred, lab = torch.argmax(outputs, axis=-1), torch.argmax(labels, axis=-1)
        
        val_loss.append(loss.item())
        val_accs.append( ((pred == lab).sum()/ len(pred)).cpu().detach().numpy() )
    
    return np.mean(val_loss), np.mean(val_accs)

In [ ]:
def train_one_epoch(model, optimizer, criterion ,train_loader, epoch):
    model.train()
    model.zero_grad()
    train_loss = []
    train_acc = []
    for i, (inputs, labels) in enumerate(train_loader): 
            
        inputs = torch.Tensor(inputs)
        inputs = inputs.type(torch.FloatTensor).to(device)
        labels = torch.Tensor(labels)
        labels = labels.type(torch.FloatTensor).to(device)
#         print(inputs.shape)
        outputs = model(inputs)
#         print(outputs.shape, labels.shape)
        loss    = criterion(outputs, labels)
        
        pred, lab = torch.argmax(outputs, axis=-1), torch.argmax(labels, axis=-1)
#         print(pred, lab)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss.append(loss.cpu().detach().numpy())
        train_acc.append( ((pred == lab).sum()/ len(pred)).cpu().detach().numpy() )
        print("Epoch {}: {}/{} Loss: {}  Acc: {}".format(epoch, i, len(train_loader), loss.item(), train_acc[-1]), end='\r')
        
    return np.mean(train_loss), np.mean(train_acc)

In [ ]:
import torch.nn as nn
class Lstm(nn.Module):
    """lstm+mlp"""
    def __init__(self, inp_dim, hidden_dim, target_size=2, num_layers=1):
        super(Lstm, self).__init__()
        self.lstm = nn.LSTM(inp_dim, hidden_dim)
        self.mlp  = nn.Linear(hidden_dim, target_size)
        
    def forward(self, x):
        _, hidden = self.lstm( x )
        x = self.mlp(x)
        return x.flatten()
    
class FNN(nn.Module):
    def __init__(self):
        super(FNN, self).__init__()
        self.fc0 = nn.Linear(2019, 1024)
        self.fc1 = nn.Linear(1024, 128)
        self.fc2 = nn.Linear(128, 32)
        self.fc3 = nn.Linear(32, 2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.4)
        
    def forward(self, x):
        x = self.fc0(x)
        x = self.relu(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [ ]:
train_config = { 'num_epochs':20, 'learning_rate':1e-4 }
arch_config  = {
                'inp_size': 90,
                'hidden_size': 180,
               }

model = Lstm(arch_config["inp_size"], arch_config["hidden_size"])
model = FNN()
model = model.to(device)
# model = torch.compile(model)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=train_config['learning_rate'], weight_decay=1e-7)

In [ ]:
train_losses = []
train_accs = []
val_losses = []
val_accs = []

In [ ]:
for epoch in range(train_config['num_epochs']):
    los, acc = train_one_epoch(model, optimizer, criterion, trainloader, epoch)
    train_losses.append(los)
    train_accs.append(acc)
    los, acc = validate(model, valloader, criterion)
    val_losses.append(los)
    val_accs.append(acc)
    print(train_losses[-1], val_losses[-1], train_accs[-1], val_accs[-1])
    if epoch % 10 == 0:
        plt.plot(train_losses, label="train loss")
        plt.plot(val_losses, label="val loss")
        plt.plot(train_accs, label="train acc")
        plt.plot(val_accs, label="val acc")
        plt.legend()
        plt.show()

In [ ]:
plt.plot(train_losses, label="train loss")
plt.plot(val_losses, label="val loss")
plt.plot(train_accs, label="train acc")
plt.plot(val_accs, label="val acc")
plt.legend()
plt.show()